<small>this tutorial is from [tensorflo.org](https://www.tensorflow.org/tutorials/quickstart/beginner)</small>

# TensorFlow 2 quickstart

## Set up TensorFlow
Import TensorFlow into your program to get started:

In [16]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.9.0-dev20211224


## Load a dataset
Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the sample data from integers to floating-point numbers:

In [17]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build a machine learning model
Build a [`tf.keras.Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) model by stacking layers.

In [18]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])
print(model)

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [19]:
predictions = model(x_train[:1]).numpy()
print("predictions: ", predictions)
print("labels: ", y_train[:1])

predictions:  [[ 0.5495028   1.104034    0.03634208  0.65336585 -0.10885612 -1.0139426
  -0.75387853 -0.3892947  -0.23573981  0.70941466]]
labels:  [5]


The [`tf.nn.softmax`](https://www.tensorflow.org/api_docs/python/tf/nn/softmax) function converts these logits to probabilities for each class:

In [20]:
tf.nn.softmax(predictions).numpy()

array([[0.13389724, 0.23313177, 0.08015096, 0.1485521 , 0.06931864,
        0.02803986, 0.03636807, 0.05236692, 0.06105836, 0.15711601]],
      dtype=float32)

Define a loss function for training using [`losses.SparseCategoricalCrossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy), which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [21]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [22]:
loss_fn(y_train[:1], predictions).numpy()

3.5741282

Before you start training, configure and compile the model using Keras [`Model.compile`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile). Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [23]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

print("x_train_shape: {}, y_train_shape: {}".format(x_train.shape, y_train.shape))
print("model_input_shape: {}, model_output_shape: {}".format(model.input_shape, model.output_shape))
model.summary()

x_train_shape: (60000, 28, 28), y_train_shape: (60000,)
model_input_shape: (None, 28, 28), model_output_shape: (None, 10)
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               100480    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


## Train and evaluate your model
Use the [`Model.fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) method to adjust your model parameters and minimize the loss:

In [24]:

model.fit(x_train, y_train, epochs=5)

Epoch 1/5


2022-07-18 16:27:08.689128: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188160000 exceeds 10% of free system memory.


1875/1875 [==============================] - 5s 3ms/step - loss: 0.2953 - accuracy: 0.9136
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1419 - accuracy: 0.9579
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1087 - accuracy: 0.9674
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0879 - accuracy: 0.9729
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0741 - accuracy: 0.9773


The [`Model.evaluate`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate) method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [27]:
model.evaluate(x_test,  y_test, verbose=2)

2022-07-18 16:30:33.185377: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 31360000 exceeds 10% of free system memory.


313/313 - 1s - loss: 0.0750 - accuracy: 0.9776 - 544ms/epoch - 2ms/step


The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [43]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

r_test = []
for r in probability_model(x_test[:5]):
    a = r.numpy().tolist()
    r_test.append(a.index(max(a)))

print("test: ", r_test)
print("real: ", y_test[:5])

test:  [7, 2, 1, 0, 4]
real:  [7 2 1 0 4]


## Conclusion
Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).